In [2]:
# Cellule notebook: MSE entre deux PNG + renormalisation explicite en [0,1]
from PIL import Image
import torch
import torch.nn.functional as F
import numpy as np

# --- Mets tes chemins ici ---
img1_path = '/home/mzoughebi/personal_study/StyleGAN2-ADA-4_Watermarking_VF/bash_launching_scripts/images_debug/generated_images/gen_img_1.png'
img2_path = '/home/mzoughebi/personal_study/StyleGAN2-ADA-4_Watermarking_VF/bash_launching_scripts/images_debug/trigger_images/trigger_img_1.png'
# ---------------------------

def load_png_renorm_01(path, mode="RGB", per_image_minmax=True, eps=1e-8):
    """
    Charge une image PNG et renormalise explicitement en [0,1].

    per_image_minmax=True  -> min-max par image (robuste si la sauvegarde a changé l'échelle)
    per_image_minmax=False -> division simple par 255 (si ton PNG est en 8-bit standard)
    """
    img = Image.open(path).convert(mode)
    arr = np.asarray(img).astype(np.float32)  # H,W,C (ou H,W)
    t = torch.from_numpy(arr)

    if t.ndim == 2:
        t = t.unsqueeze(-1)  # H,W,1

    # H,W,C -> C,H,W
    t = t.permute(2, 0, 1)

    if per_image_minmax:
        t_min = t.amin()
        t_max = t.amax()
        t = (t - t_min) / (t_max - t_min + eps)
    else:
        # cas PNG 8-bit classique
        t = t / 255.0

    return t.unsqueeze(0)  # [1,C,H,W]

x = load_png_renorm_01(img1_path, per_image_minmax=True)
y = load_png_renorm_01(img2_path, per_image_minmax=True)

assert x.shape == y.shape, f"Tailles différentes: {x.shape} vs {y.shape}"

mse = F.mse_loss(x, y).item()
print("MSE (après renorm [0,1]):", mse)


MSE (après renorm [0,1]): 0.07762851566076279
